In [ ]:
#!jupyter nbconvert --to=python DIN_WOA_Comparison_python3.ipynb

# After changes in the script using Jupyter notebook, save the notebook, run this line, and clear the output.

In [ ]:
class DINcomp:
    '''
    class DINcomp
    
    compare FESOM-REcoM DIN data to World Ocean Atlas 
    
    -use layerwise = True to compare a set of depth
    -use depth_limit to specify maximum depth for mean-over-depth comparison
    '''
    def __init__(self,runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                 WOAvar='n_an',
                 mapproj='pc',
                 cmap = 'viridis',
                 savefig=False,
                 layerwise=False,depth_array=[],
                 depth_limit=100):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.WOAvar = WOAvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.depth_limit = depth_limit

        import matplotlib.pyplot as plt
        import numpy as np
        #from netCDF4 import Dataset
        #from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf

        from load_interp_WOA_python3 import WOAdata
        from plot_Taylor_normalized import plt_Taylor_norm

        # load FESOM data -------------------------------------------------------------------------------------
        print('Processing ' + self.resultpath)
        
        mesh       = pf.load_mesh(self.meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)

        # check variables
        #NCfesom = self.resultpath + '/DIN.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM DIN {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'DIN [mmol m$^{-3}$]' # equals to mumol/L

        # load data -------------------------------------------------------------------------------------
        DINfesom = pf.get_data(self.resultpath, "DIN", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)

        self.DINfesom = DINfesom

        # load WOA data  -------------------------------------------------------------------------------------
        WOA_input = WOAdata(self.runname,self.resultpath,self.meshpath,self.ncpath,self.WOAvar, get_overview=False)
        din_int = WOA_input.woa_int    
        #print(np.shape(din_int))
        
        labelwoa = 'World Ocean Atlas DIN'
        unitwoa = 'DIN [$\mu$mol L$^{-1}$]'

        # apply sea mask to WOA as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        din_int_ma = np.copy(din_int)
        din_int_ma[DINfesom == 0] = 0
        
        self.din_int = din_int
        self.din_int_ma = din_int_ma

        # plot WOA and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)

            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

                if(True):
                    print('\nInput depth = {0}, plotting at depth = {1} m\nFESOM min = {2}, max = {3}\nWOA min = {4}, max = {5}'.format(
                        d,plot_depth,
                        np.nanmin(DINfesom[:,i]),np.nanmax(DINfesom[:,i]),
                        np.nanmin(din_int_ma[:,i]),np.nanmax(din_int_ma[:,i])))
                    print('Min of difference WOA-FESOM: {0}'.format(np.nanmin(din_int_ma[:,i]-DINfesom[:,i])))

                pf.plot(mesh, [din_int_ma[:,i],DINfesom[:,i], din_int_ma[:,i]-DINfesom[:,i]], 
                        rowscol= (1,3),
                        levels = (0,45,40),
                        units=unitwoa + '\n at depth = {0} m'.format(plot_depth), 
                        mapproj=self.mapproj, # robinson projection takes more time!
                        cmap = self.cmap,
                        titles=[labelwoa, labelfesom, 'WOA - FESOM'],figsize = (20,20)
                       )

            # statistics  -------------------------------------------------------------------------------------
            # preparation of datasets
            if np.isnan(np.min(din_int)): print('WARNING: The interpolated WOA field contains NaNs at depth')
            if np.isnan(np.min(DINfesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')


            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

                title = 'Taylor Diagram for DIN at {0} m'.format(plot_depth)
                plt_Taylor_norm(din_int[i,:],DINfesom[i,:],mask=True,title=title)

                if False:
                    # get statistics only from ocean gridpoints (same mask assumption as above)
                    ind_stat = np.where(DINfesom[i,:] != 0)

                    taylor_stats1 = sm.taylor_statistics(din_int[i,ind_stat],DINfesom[i,ind_stat])
                    sdev = np.array([taylor_stats1['sdev'][0], taylor_stats1['sdev'][1]])
                    crmsd = np.array([taylor_stats1['crmsd'][0], taylor_stats1['crmsd'][1]])
                    ccoef = np.array([taylor_stats1['ccoef'][0], taylor_stats1['ccoef'][1]])

                    label = ['Observation', 'Model']# at {0} m'.format(plot_depth)]

                    fig = plt.figure(figsize=(7,7), facecolor='w', edgecolor='k')
                    sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                                          titleOBS = 'observation', markerLabel = label,
                                          markerLabelColor = 'c',
                                          markerColor = 'c', markerLegend = 'on',
                                          tickRMS = range(0,5,1), tickRMSangle = 135.0,
                                          colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                                          titleRMS = 'off', tickSTD = range(0,10,2 ),
                                          axismax = 10.0, 
                                          colSTD = 'b', styleSTD = '-.',
                                          widthSTD = 1.0, titleSTD = 'on',
                                          colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                                          titleCOR = 'on')
                    plt.title('Taylor Diagram for DIN at {0} m'.format(plot_depth), loc='right')
        
            if(self.savefig==True): print('\n***\n***Too many figures to export...\n***')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            # get level depth index closest to depth_limit
            i_depth_limit = pf.ind_for_depth(self.depth_limit, mesh)
            
            # corresponding level depth
            depth_limit_level = - mesh.zlev[i_depth_limit]
            
            # corresponding layer depth (where concentrations are defined):
            depth_limit_layer = - WOA_input.layer_depths[i_depth_limit]
            
            # print overview of chosen depth
            print('DIN as mean over depth \nwith max layer depth = {0} \n(level depth = {1}m, mesh index {2})'.format(
                depth_limit_layer,depth_limit_level,i_depth_limit))
            
            # mean over depth
            DINfesom_mean = np.mean(DINfesom[:,:i_depth_limit], axis = 1)
            din_int_ma_mean = np.mean(din_int_ma[:,:i_depth_limit], axis = 1)
            #print(np.shape(DINfesom_mean), np.shape(din_int_ma_mean))
            
            self.DINfesom_mean = DINfesom_mean
            self.din_int_ma_mean = din_int_ma_mean

            print('\nFESOM min = {0}, max = {1}\nWOA min = {2}, max = {3}'.format(
                    np.nanmin(DINfesom_mean),np.nanmax(DINfesom_mean),
                    np.nanmin(din_int_ma_mean),np.nanmax(din_int_ma_mean),
                    depth_limit_layer))
            print('Min of difference WOA-FESOM: {0}'.format(np.nanmin(din_int_ma_mean-DINfesom_mean)))
    
            fig_data = pf.plot(mesh, [din_int_ma_mean,DINfesom_mean, din_int_ma_mean-DINfesom_mean], 
                    rowscol= (1,3),
                    levels = (-15,35,51),
                    units=unitwoa, 
                    mapproj=self.mapproj, # robinson projection takes more time!
                    cmap = self.cmap,
                    titles=['World Ocean Atlas: Interpolated DIN for initialization', 
                            'Mean over depth (max={2}m)\nFESOM Dissolved Inorganic Nitrogen ({0}-{1})'.format(
                                first_year,last_year,depth_limit_layer),
                            'WOA - FESOM'],
                    figsize = (20,20)
                   )
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'DIN_WOA'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
            plt.show(block=False)  
                
            # statistics  -------------------------------------------------------------------------------------            
            # preparation of datasets
            if np.isnan(np.min(din_int_ma_mean)): print('WARNING: The interpolated WOA field contains NaNs at depth')
            if np.isnan(np.min(DINfesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

            title = 'Taylor Diagram for DIN \n(mean over depth, max = {0}m)'.format(depth_limit_layer)
            plt_Taylor_norm(din_int_ma_mean,DINfesom_mean,mask=True,title=title,verbose=True)
                
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):                
                plt.savefig(self.savepath+self.runname+'_'+'DIN_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
            plt.show(block=False)  
        


In [ ]:
### TESTING ###
if __name__ == "__main__":
    #from DIN_WOA_Comparison_python3 import DINcomp
    
    # run specification -------------------------------------------------------------------------------------
    runid      =  'fesom'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + 'mo20'
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

    # period of analysis ------------------------------------------------------------------------------------

    first_year = 1965
    last_year  = 1966

    # WOA ------------------------------------------------------------------------------------    
    ncpath                = '/work/ollie/projects/MarESys/evaluation/woa13_all_n00_01.nc'
    #!ncdump -h $ncpath
    
    # now test:
    test = DINcomp(runid,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                   mapproj = 'pc', cmap = 'viridis',
                   depth_limit = 200,
                   layerwise=False,savefig=False)

In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    from load_interp_WOA_python3 import WOAdata
    
    WOAdata?
    # -> get docstring